<a href="https://www.kaggle.com/code/aisuko/fine-tuning-t5-small-with-lora?scriptVersionId=165063699" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Overview

**Note: All the images are from the Reference lists section or the internet**

We are going to fine-tuning a t5-small quantized model for English to French translation-related tasks using the LoRA method. We use the peft and transformers libraries for training.


# About quantization

Please check the list below:

* [Quantization Technologies](https://www.kaggle.com/code/aisuko/quantization-technologies)
* [Zero Degradation matrix multiplication](https://www.kaggle.com/code/aisuko/zero-degradation-matrix-multiplication)
* [Lighter models on GPU for inference](https://www.kaggle.com/code/aisuko/lighter-models-on-gpu-for-inference)


# About LoRA(Low Rank Adaptation)

> A technique that accelerates the fine-tuning of large models while consuming less memory.


**The idea is to freeze the original pre-trained weights(Matrices) and introduce new updata matrices**. These new matrics are trained on new data while keeping the overall number of changes low. The original weights matrix doesn't receive any adjustments. And finally, both the original and the adapted weights are combined.

![](https://files.mastodon.social/media_attachments/files/111/702/004/494/881/797/original/a26697e010f0096b.webp)

LoRA makes fine-tuning more efficient by drastically reducing the number of **trainable parameters**. In principle, LoRA can be applied to any subset of weight matrices in a neural network to reduce the number of trainable parameters. However, for simplicity and further parameter efficiency, **in Transformer models LoRA is typically applied to attention blocks only**. **The resulting number of trainable parameters in a LoRA model depends on the size of the low-rank update matrices, which is determined manily by the rank `r` and the shape of the original weight matrix**.


The differences between QLoRA and LoRA in real word case see notebook [fine-tuning llama2 with QLoRA](https://www.kaggle.com/code/aisuko/fine-tuning-llama2-with-qlora?scriptVersionId=158763163&cellId=1).

In [1]:
%%capture
!pip install transformers==4.36.2
!pip install bitsandbytes==0.41.3
!pip install accelerate==0.25.0
!pip install datasets==2.15.0
!pip install evaluate==0.4.1
!pip install peft==0.7.1

In [2]:
import os
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))

os.environ["WANDB_API_KEY"]=user_secrets.get_secret("WANDB_API_KEY")
os.environ["WANDB_PROJECT"] = "Fine-tuning t5-small-on-opus100"
os.environ["WANDB_NAME"] = "ft-t5-small-on-opus100"

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Loading Dataset

We are going to use the optus 100 dataset for training which gives us access to more than 100 different languages.

In [3]:
from datasets import get_dataset_config_names

configs=get_dataset_config_names("opus100")
print(configs)

['af-en', 'am-en', 'an-en', 'ar-de', 'ar-en', 'ar-fr', 'ar-nl', 'ar-ru', 'ar-zh', 'as-en', 'az-en', 'be-en', 'bg-en', 'bn-en', 'br-en', 'bs-en', 'ca-en', 'cs-en', 'cy-en', 'da-en', 'de-en', 'de-fr', 'de-nl', 'de-ru', 'de-zh', 'dz-en', 'el-en', 'en-eo', 'en-es', 'en-et', 'en-eu', 'en-fa', 'en-fi', 'en-fr', 'en-fy', 'en-ga', 'en-gd', 'en-gl', 'en-gu', 'en-ha', 'en-he', 'en-hi', 'en-hr', 'en-hu', 'en-hy', 'en-id', 'en-ig', 'en-is', 'en-it', 'en-ja', 'en-ka', 'en-kk', 'en-km', 'en-kn', 'en-ko', 'en-ku', 'en-ky', 'en-li', 'en-lt', 'en-lv', 'en-mg', 'en-mk', 'en-ml', 'en-mn', 'en-mr', 'en-ms', 'en-mt', 'en-my', 'en-nb', 'en-ne', 'en-nl', 'en-nn', 'en-no', 'en-oc', 'en-or', 'en-pa', 'en-pl', 'en-ps', 'en-pt', 'en-ro', 'en-ru', 'en-rw', 'en-se', 'en-sh', 'en-si', 'en-sk', 'en-sl', 'en-sq', 'en-sr', 'en-sv', 'en-ta', 'en-te', 'en-tg', 'en-th', 'en-tk', 'en-tr', 'en-tt', 'en-ug', 'en-uk', 'en-ur', 'en-uz', 'en-vi', 'en-wa', 'en-xh', 'en-yi', 'en-yo', 'en-zh', 'en-zu', 'fr-nl', 'fr-ru', 'fr-zh', 

We will use the "en-fr" data for language translation. Let's download and load the dataset though the `load_dataset`.

In [4]:
from datasets import load_dataset

dataset=load_dataset("opus100", "en-fr")
dataset

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 1000000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
})

# Data Tokenization

In [5]:
from transformers import AutoTokenizer

model_name="t5-small"
prompt="My name is Kaggle, nice to see you."

tokenizer=AutoTokenizer.from_pretrained(model_name, load_in_8bit=True, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [6]:
# use a sample of around 2000 instead of the complete dataset as training dataset
train_dataset=dataset['train'].shuffle(seed=42).select(range(2000))

# as evaluation dataset
eval_dataset=dataset['validation']


def preprocess_func(data):
    inputs=[ex['en'] for ex in data['translation']]
    targets=[ex['fr'] for ex in data['translation']]
    
    # tokenize each row of inputs and outputs
    model_inputs=tokenizer(inputs, truncation=True)
    labels=tokenizer(targets, truncation=True)
    
    model_inputs["labels"]=labels["input_ids"]
    return model_inputs


# We tokenize the entire dataset
train_dataset=train_dataset.map(preprocess_func, batched=True)
eval_dataset=eval_dataset.map(preprocess_func, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

# Preparing the model

First, we will load the model with 8 bit(quantization the model). And then, we are using the LoRA. Here are the description of the LoraConfig:

* **r**: the rank of the update matrices, expressed in **int**. Lower rank results in smaller update matrices with fewer trainable parameters.
* **target_modules**: **The modules(for example, attention blocks)** to apply the LoRA update matrices.
* **alpha**: LoRA scaling factor
* **bias**: Specifies if the bias parameters should be trained. Can be 'none','all' or 'lora_only'.
* **module_to_save**: List of modules apart from LoRA layers to be set as trainable and saved in the final checkpoint. These typically include model's custome head that is randomly initialized for the fine-tuning task.
* **layer_to_transform**: List of layers to be transformed by LoRA. If not specified, all layers in `target_modules` are transformed.
* **layers_pattern**: Pattern to match layer names in `target_modules`, if `layer_to_transform` is specified. By default `PeftModel` will look at common layer pattern(`layers`,`h`, `blocks`, etc.), use it for exotic and custom models.
* **rank_pattern**: The mapping from layer names or regexp expression to ranks which are different from the default tank specified by `r`.
* **alpha_pattern**: The mapping from layer names or regexp expression to alphas which are different from the default alpha specified by `lora_alpha`.

In [7]:
from peft import PeftModel, prepare_model_for_kbit_training, PeftConfig, get_peft_model, LoraConfig, TaskType
from transformers import BitsAndBytesConfig
from transformers import AutoModelForSeq2SeqLM

bnb_config=BitsAndBytesConfig(
    load_in_8bit=True
)

model=AutoModelForSeq2SeqLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [8]:
# Freeze the original parameters
model=prepare_model_for_kbit_training(model)

peft_config=LoraConfig(
    # the task to train for (sequence-to-sequence language modeling in this case)
    task_type=TaskType.SEQ_2_SEQ_LM,
    # the dimension of the low-rank matrices
    r=4,
    # the scaling factor for the low-rank matrices
    lora_alpha=32,
    # the dropout probability of the LoRA layers
    lora_dropout=0.01,
    target_modules=["k","q","v","o"],
)

peft_model=get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 294,912 || all params: 60,801,536 || trainable%: 0.4850403779272945


In [9]:
import evaluate
import numpy as np

accuracy=evaluate.load("accuracy")

def compute_metrics(p):
    predictions, labels=p
    predictions=np.argmax(predictions, axis=1)
    
    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

# Ttraining

In [10]:
from transformers import DataCollatorForSeq2Seq

# ignore tokenizer pad token in the loss
label_pad_token_id=-100

# padding the sentence of the entire datasets
data_collator=DataCollatorForSeq2Seq(
    tokenizer=tokenizer, 
    model=peft_model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [11]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, Trainer
import torch

training_args=Seq2SeqTrainingArguments(
    output_dir=os.getenv("WANDB_NAME"),
    auto_find_batch_size=True,
    learning_rate=1e-3,
    num_train_epochs=2,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_dir=os.getenv("WANDB_NAME")+"/logs",
    logging_strategy="epoch",
    logging_steps=500,
    save_strategy="epoch",
    push_to_hub=False,
    report_to="wandb",
    run_name=os.getenv("WANDB_NAME"),
)


# Create Trainer instance
trainer=Seq2SeqTrainer(
    model=peft_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

peft_model.config.use_cache=False
trainer.train()

wandb: Currently logged in as: urakiny (causal_language_trainer). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240301_224634-aoycmpgl
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ft-t5-small-on-opus100
wandb: ⭐️ View project at https://wandb.ai/causal_language_trainer/Fine-tuning%20t5-small-on-opus100
wandb: 🚀 View run at https://wandb.ai/causal_language_trainer/Fine-tuning%20t5-small-on-opus100/runs/aoycmpgl
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cas

Step,Training Loss
63,1.673500
126,1.471900


/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


TrainOutput(global_step=126, training_loss=1.5727293831961495, metrics={'train_runtime': 126.9061, 'train_samples_per_second': 31.519, 'train_steps_per_second': 0.993, 'total_flos': 111144119304192.0, 'train_loss': 1.5727293831961495, 'epoch': 2.0})

In [12]:
# trainer.evaluate() # out of GPU memory

In [13]:
kwargs={
    'model_name': f'{os.getenv("WANDB_NAME")}',
    'finetuned_from': model_name,
    'tasks': 'Translation',
#     'dataset_tags':'',
    'dataset':'opus100'
}

tokenizer.push_to_hub(os.getenv("WANDB_NAME"))
trainer.push_to_hub(**kwargs)

adapter_model.safetensors:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

'https://huggingface.co/aisuko/ft-t5-small-on-opus100/tree/main/'

# Inference

In [14]:
peft_model.config.use_cache=True
context=tokenizer(["Floods in Australia 2024"], return_tensors="pt")
output=peft_model.generate(**context)

tokenizer.decode(output[0], skip_special_tokens=True)

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1636: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `inp

'Inondations en Australie 2024 2024 2024 2024 2024 20'

In [15]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
model = PeftModel.from_pretrained(model, "aisuko/ft-t5-small-on-opus100")

output=model.generate(**context)
tokenizer.decode(output[0], skip_special_tokens=True)

adapter_config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

'Inondations en Australie 2024 2024 2024 2024 2024 20'

# References List

* https://towardsdev.com/fine-tune-quantized-language-model-using-lora-with-peft-transformers-on-t4-gpu-287da2d5d7f1
* https://huggingface.co/docs/peft/quicktour